In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage , AIMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
import yaml
from yaml.loader import SafeLoader
from RAG import return_rag_chain

from elevenlabs.client import ElevenLabs
from elevenlabs import play
import os
import pyaudio
import wave
import keyboard  # For keypress detection

# Parameters
FORMAT = pyaudio.paInt16  # Audio format (16-bit)
CHANNELS = 1  # Mono audio
RATE = 16000  # Sampling rate (16 kHz)
CHUNK = 1024  # Number of frames per buffer

In [2]:
import whisper


In [3]:
model = whisper.load_model("base.en")  # Options: "tiny", "base", "small", "medium", "large"

def transcribe_long_audio(file_path):
    result = model.transcribe(file_path)
    return result["text"]

# Example usage
audio_file = "Record (online-voice-recorder.com).mp3"
transcribed_text = transcribe_long_audio(audio_file)
print("Transcribed Text:", transcribed_text)

c:\Users\Hossa\miniconda3\envs\py310\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

Transcribed Text:  Hello, can you tell me what is the best source of protein in order to increase my muscles?


In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [5]:
import requests
def get_transcription():
    # API endpoint
    url = "https://api.groq.com/openai/v1/audio/transcriptions"

    # Your Groq API key
    api_key = "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"

    # Path to your audio file
    audio_file_path = "output.wav"

    # Headers
    headers = {
        "Authorization": f"Bearer {api_key}"
    }

    # Form data
    files = {
        "file": open(audio_file_path, "rb")
    }
    data = {
        "model": "whisper-large-v3",
        "language": "en"
    }

    # Send the request
    response = requests.post(url, headers=headers, files=files, data=data)

    # Check the response
    if response.status_code == 200:
        # Success
        transcription = response.json()
        return transcription["text"]
    else:
        # Error
        print("Error:", response.status_code, response.text)

In [6]:
from elevenlabs import  stream


In [7]:
with open('config.yaml') as file:
    config = yaml.load(file, Loader=SafeLoader)
elevenlabs_api_key = config['elevnlabs']['apiKey']

In [13]:

vectorstore.save_local("nuitrionsDB")

In [17]:
os.environ["ELEVENLABS_API_KEY"] = elevenlabs_api_key

In [18]:
client = ElevenLabs(api_key=elevenlabs_api_key)

In [19]:
rag=return_rag_chain()

Vectorstore created successfully


In [20]:
response=rag.invoke({"input": "tell what is the best way to increase my gym session time as i get tired so fast.", "chat_history": []})


In [21]:
def print_responses(responses):
    for response in responses:
        print(response, end="",flush=True)

In [ ]:

rag_chain=rag.pick('answer')


In [23]:
response=rag_chain.stream({"input": "tell what is the best way to increase my gym session time as i get tired so fast.", "chat_history": []})

In [30]:

def record_audio():
# Initialize PyAudio
    audio = pyaudio.PyAudio()
    
    # Open audio stream
    stream = audio.open(
        format=FORMAT,
        channels=CHANNELS,
        rate=RATE,
        input=True,
        frames_per_buffer=CHUNK
    )
    
    print("Recording... Press 'Enter' to stop.")
    
    frames = []
    is_recording = True
    
    # Start recording
    while is_recording:
        # Read audio data from the stream
        data = stream.read(CHUNK)
        frames.append(data)
        
        # Check if the 'Enter' key is pressed
        if keyboard.is_pressed('enter'):
            is_recording = False
    
    print("Recording stopped.")
    
    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()
    
    # Save the recorded audio to a file
    with wave.open("output.wav", "wb") as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b"".join(frames))
    
    print("Audio saved to output.wav")

In [37]:
flag=True
chat_history = InMemoryChatMessageHistory()

while flag:
    record_audio()
    question = get_transcription()
    response = rag.invoke({"input": question, "chat_history": chat_history})["answer"]
    
    audio = client.text_to_speech.convert(
        text=response,
        voice_id="JBFqnCBsd6RMkjVDRZzb",
        model_id="eleven_flash_v2",
        output_format="mp3_44100_128",
    )
    play(audio)
    chat_history.append(HumanMessage(content=question))
    chat_history.append(AIMessage(content=response))
    flag=False
    print("Press 'space' to restart or 'Esc' to exit...")
    if keyboard.is_pressed('space'):
        flag=True
    elif keyboard.is_pressed('esc'):
        break
    



Recording... Press 'Enter' to stop.
Recording stopped.
Audio saved to output.wav


ValueError: variable chat_history should be a list of base messages, got  of type <class 'langchain_core.chat_history.InMemoryChatMessageHistory'>